In [ ]:
import datetime as dt
import sys
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
import pandas as pd
# from pandas_datareader import data as web
import seaborn as sns
from pylab import rcParams 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from arch import arch_model
from numpy.linalg import LinAlgError
from scipy import stats
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from sklearn.metrics import mean_squared_error
from scipy.stats import probplot, moment
from arch import arch_model
from arch.univariate import ConstantMean, GARCH, Normal
from sklearn.model_selection import TimeSeriesSplit
from datasets import load_dataset
import warnings

import os
from pathlib import Path
import pandas as pd
from datasets import load_dataset  # Assuming this is from the 'datasets' library
import sys
from pathlib import Path

# Add the parent directory of the notebook to the Python path
sys.path.append(str(Path().resolve().parent))


from funcs.data_read import (
    read_txn_data,  
    read_ob_data,
    load_ob_features,
    load_trx_features,
)

from funcs.preprocess import (
    preprocess_ob_data, 
    preprocess_trx_data,
)

from funcs.features import (
    create_features_order_book,
    create_features_trx,
)

from funcs.utils import (
    filter_extremes,
    deseason_total_volume,
)



In [6]:
trx_1min_features = load_trx_features("1min")
ob_1min_fatures=load_ob_features(freq="1min")

✅ Loaded TRX features (1min) from:
c:\Users\peter\codes\ml\eth\Komplex\trx_features\trx_features_1min.parquet
Reading 109 files from c:\Users\peter\codes\ml\eth\Komplex\ob_features\1min...
Combined DataFrame for 1min created. Total rows: 155,954


In [9]:
# Merge the two DataFrames on "timestamp_id" ,max diff is 1  min
 
# Ensure both DataFrames have the same timestamp format
trx_1min_features['timestamp_id'] = pd.to_datetime(trx_1min_features['timestamp_id'])
ob_1min_fatures['timestamp_id'] = pd.to_datetime(ob_1min_fatures['timestamp_id'])

# Merge the DataFrames on "timestamp_id" with a tolerance of 2 minutes
merged_df = pd.merge_asof(
    trx_1min_features.sort_values("timestamp_id"),
    ob_1min_fatures.sort_values("timestamp_id"),
    on="timestamp_id",
    direction="nearest",
    tolerance=pd.Timedelta("2min")
)

# Check the result of the merge
print(f"Merged DataFrame created. Total rows: {len(merged_df):,}")
print(merged_df)

Merged DataFrame created. Total rows: 168,743
                        timestamp_id                datetime_x  first_price  \
0      1970-01-01 00:00:01.527800400 2018-05-31 21:00:00+00:00      7538.34   
1      1970-01-01 00:00:01.527800460 2018-05-31 21:01:00+00:00      7532.43   
2      1970-01-01 00:00:01.527800520 2018-05-31 21:02:00+00:00      7535.01   
3      1970-01-01 00:00:01.527800580 2018-05-31 21:03:00+00:00      7540.19   
4      1970-01-01 00:00:01.527800640 2018-05-31 21:04:00+00:00      7541.00   
...                              ...                       ...          ...   
168738 1970-01-01 00:00:01.538344500 2018-09-30 21:55:00+00:00      6553.46   
168739 1970-01-01 00:00:01.538344560 2018-09-30 21:56:00+00:00      6555.77   
168740 1970-01-01 00:00:01.538344620 2018-09-30 21:57:00+00:00      6555.75   
168741 1970-01-01 00:00:01.538344680 2018-09-30 21:58:00+00:00      6555.00   
168742 1970-01-01 00:00:01.538344740 2018-09-30 21:59:00+00:00      6555.73   

     

In [ ]:
merged_df

